In [1]:
import tensorflow as tf
myLogdet_mod = tf.load_op_library('./myLogdet_op.so')

In [ ]:
from tensorflow.python.framework import ops


@ops.RegisterGradient("MyLogdet")
def _my_logdet_grad(op, grad1, grad2):
    return [tf.multiply(grad1, op.outputs[1])]

In [7]:
import numpy as np
#import time
np.random.seed(0)
numMat = 777
dim = 100
b = np.zeros(shape=(numMat, dim, dim))
for idx in range(numMat):
    a = np.random.randn(dim,dim)
    b[idx] = 0.01 * a.dot(a.T) + np.eye(dim)
b = np.array(b, dtype='float32')
A = tf.constant(b)

In [8]:
with tf.device('/gpu:0'):
    L_gpu = myLogdet_mod.my_logdet(A)
    
sess = tf.Session()

In [14]:
%%time
Ag_gpu = sess.run(L_gpu)

CPU times: user 39.4 ms, sys: 11 ms, total: 50.3 ms
Wall time: 46.6 ms


In [ ]:
print Ag_gpu[0]
print np.log(np.linalg.det(b))

print np.max(np.abs(Ag_gpu[0] - np.log(np.linalg.det(b))))
print np.max(np.abs(Ag_gpu[1] - np.linalg.inv(b)))

In [ ]:
with tf.device('/cpu:0'):
    L_cpu = tf.log(tf.matrix_diag_part(tf.cholesky(A)))
    Lc_loss = 2 * tf.reduce_sum(L_cpu)
    Ac_grad = tf.gradients(Lc_loss, [A])[0]
    
sess = tf.Session()

In [ ]:
%%time
Ag_cpu = sess.run(Ac_grad)

In [ ]:
np.max(np.abs(Ag_gpu - Ag_cpu))

In [ ]:
Ag_gpu - Ag_cpu